<a href="https://colab.research.google.com/github/dejunga/ML-Vehicle_Image_Classification/blob/main/train_custom_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import pickle

from google.colab import drive
drive.mount('/content/drive')

# Set up directories for training data
main_dir = "/content/drive/My Drive/Vehicles"
categories = ["Auto Rickshaws", "Bikes", "Cars", "Motorcycles", "Planes", "Ships", "Trains"]
base_split_dir = "/content/drive/My Drive/vehicles_split"

# Custom neural network model with Batch Normalization and enhanced architecture
model = Sequential()

# First convolutional layer (smaller filter size) + Batch Normalization
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

# Second convolutional layer + Batch Normalization
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

# Third convolutional layer + Batch Normalization
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

# Flattening the output for fully connected layers
model.add(Flatten())

# Fully connected layer with fewer neurons + Dropout
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))

# Output layer for classification
model.add(Dense(len(categories), activation='softmax'))

# Compile the model using Adam optimizer with a reduced learning rate
optimizer = Adam(learning_rate=1e-4)  # You can experiment with different learning rates
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Create data generators with additional augmentations
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,  # Increased zoom augmentation
    horizontal_flip=True,
    fill_mode='nearest'  # Filling in missing pixels after augmentation
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

# Ensure the directories exist before creating generators
train_generator = train_datagen.flow_from_directory(os.path.join(base_split_dir, "train"),
                                                    target_size=(150, 150), batch_size=32, class_mode='categorical')
val_generator = val_test_datagen.flow_from_directory(os.path.join(base_split_dir, "val"),
                                                     target_size=(150, 150), batch_size=32, class_mode='categorical')

# Callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Train the model and save the history
history = model.fit(train_generator, epochs=10, validation_data=val_generator,
                    callbacks=[early_stopping, reduce_lr])  # Reduced learning rate if the validation loss plateaus

# Save the training history
history_save_path = '/content/drive/My Drive/models/custom_model_history.pkl'
with open(history_save_path, 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

# Save the custom model
model_save_path = '/content/drive/My Drive/models/custom_vehicle_model.keras'
model.save(model_save_path)

print(f"Custom model trained and saved to {model_save_path} successfully!")

Mounted at /content/drive
Found 3911 images belonging to 7 classes.
Found 838 images belonging to 7 classes.
Epoch 1/10
